This notebook aims to build a graph purely with KNN.

In [2]:
import numpy as np
from nuscenes.nuscenes import NuScenes

# nusc = NuScenes(version='v1.0-mini', dataroot='/media/HDD2/Datasets/mini_nusc', verbose=True)

nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/nuscenes2', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 48.572 seconds.
Reverse indexing ...
Done reverse indexing in 10.4 seconds.


: 

In [2]:
import torch_geometric as tgeo

tgeo.get_home_dir()

'/home/maximilian/.cache/pyg'

In [3]:
from nuscenes.utils.splits import train,val
from nuscenes.utils.splits import create_splits_scenes

split = create_splits_scenes()
print(split.keys())
split_scene_list = []
for scene_name in split['mini_train']:
    for scene in nusc.scene:
        if scene['name']==scene_name:
            split_scene_list.append(scene)

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])


In [5]:
print(split['mini_train'])
print(split['mini_val'])

['scene-0061', 'scene-0553', 'scene-0655', 'scene-0757', 'scene-0796', 'scene-1077', 'scene-1094', 'scene-1100']
['scene-0103', 'scene-0916']


In [5]:
#write a sequence 

sample_dict = {}

i = 0 
for scene in split_scene_list:
    last_sample_token =""
    sample_token = scene['first_sample_token']
    while(last_sample_token == ""):
        
        sample = nusc.get('sample', sample_token)
        sample_dict[i] = (scene['token'],sample["token"])
        i += 1
        sample_token = sample["next"]
        if(sample["token"]== scene['last_sample_token']):
            last_sample_token = scene['last_sample_token']

print (len(sample_dict))
print (sample_dict[0])
print (sample_dict[1])
print (sample_dict[2])

323
('cc8c0bf57f984915a77078b10eb33198', 'ca9a282c9e77460f8360f564131a8af5')
('cc8c0bf57f984915a77078b10eb33198', '39586f9d59004284a7114a68825e8eec')
('cc8c0bf57f984915a77078b10eb33198', '356d81f38dd9473ba590f39e266f54e5')


In [6]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.is_available())
    num_gpu = torch.cuda.device_count()
    print("num_gpu:",num_gpu)
    print("current_device: ",torch.cuda.current_device())
    print("---------------------------------------------------------------")
    for i in range(num_gpu):
        print(torch.cuda.get_device_name(i))
        # print(torch.cuda.device(i))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB', torch.cuda.memory_allocated(i),torch.cuda.memory_allocated(i)/1024**3)
        print('Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB', torch.cuda.memory_reserved(i),torch.cuda.memory_reserved(i)/1024**3)
        print("---------------------------------------------------------------")

Using device: cuda
True
num_gpu: 2
current_device:  0
---------------------------------------------------------------
GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------
GeForce RTX 2080
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------


In [7]:
import torch
import numpy as np

# from enum import IntEnum
# class edge_types(IntEnum):
#     spatial_edges = 0
#     temporal_edges = 1

# y = torch.LongTensor(4,1).random_() % 2
# print("y:", y)
# y_onehot = torch.FloatTensor(4, 2)
# y_onehot.zero_()
# y_onehot.scatter_(1,y,1)
# print("y_onehot:", y_onehot)

# from sklearn.neighbors import BallTree
# rng = np.random.RandomState(0)
# X = rng.random_sample((10, 3))
# tree = BallTree(X, leaf_size=2) 
# dist, ind = tree.query(X, k=3) 
# print(ind)  # indices of 3 closest neighbors
# ind = ind[:,1:]
# print(ind) 

In [8]:
#Create List of Graph objects
from datasets.nuscenes_mot_graph import NuscenesMotGraph
#______________________________________________________________#
# Decide if only first scene should be computed
only_first_scene = True
scene_token0, sample_token0= sample_dict[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    for i in range(torch.cuda.device_count()):
        if torch.cuda.get_device_name(i) == "GeForce RTX 2080":
            device = torch.device(i) # RTX 2080 8GB
print("Device:", device)
#_______________________________________________________________#

MotGraphList= []
for sample_key in sample_dict:
    scene_token_current, sample_token_current= sample_dict[sample_key]
    if(only_first_scene):
        if(scene_token0 == scene_token_current):
            object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3, 
                        filterBoxes_categoryQuery='vehicle.car',
                        device= device)
            is_possible2construct = object.is_possible2construct
            if is_possible2construct:
                object.construct_graph_object()
                # object.assign_edge_labels_one_hot()
                object.assign_edge_labels(label_type='multiclass')
                # object.assign_edge_labels(label_type='binary')
                MotGraphList.append(object)
    else:
        object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3,  
                        filterBoxes_categoryQuery='vehicle.car',device= device)
        is_possible2construct = object.is_possible2construct
        if is_possible2construct:
            object.construct_graph_object()
            # object.assign_edge_labels(label_type='binary')
            object.assign_edge_labels(label_type='multiclass')
            MotGraphList.append(object)
            

Device: cuda:1


In [9]:
# first_graph = MotGraphList[0]
# last_graph = MotGraphList[-1]
# first_graph_object = first_graph.graph_obj
# last_graph_object = last_graph.graph_obj

# print("Node features: \n",first_graph_object.x)
# print("Node features: \n ",last_graph_object.x)
# print("----------------------------------------\n")
# print("Edge indices \n", first_graph_object['edge_index'])
# print("Edge indices \n", last_graph_object['edge_index'])
# print("----------------------------------------\n")
# print("Edge attributes \n", first_graph_object['edge_attr'])
# print("Edge attributes \n", last_graph_object['edge_attr'])
# print("----------------------------------------\n")
# print("temporal_edges_mask \n", first_graph_object.temporal_edges_mask)
# print("temporal_edges_mask \n", last_graph_object.temporal_edges_mask)
# print("----------------------------------------\n")
# print("Edge labels: one hot encoding \n",first_graph_object["edge_labels"])
# print("Edge labels: one hot encoding \n",last_graph_object["edge_labels"])

In [19]:
# Input
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj.keys)
print("----------------------------------------\n")
print("Node features: ", MotGraphList[0].graph_obj['x'])
print("----------------------------------------\n")
print("Edge indices", MotGraphList[0].graph_obj['edge_index'])
print("----------------------------------------\n")
print("Edge attributes", MotGraphList[0].graph_obj['edge_attr'])

----------------------------------------

['edge_attr', 'edge_index', 'x', 'edge_labels', 'temporal_edges_mask']
----------------------------------------

Node features:  tensor([[ 3.7352e+01,  6.4397e+01,  4.5099e-01,  0.0000e+00],
        [ 9.1482e+00, -1.9542e+01, -1.6450e+00,  0.0000e+00],
        [ 5.9793e+00,  3.5009e+01,  4.4059e-02,  0.0000e+00],
        [-8.2717e+00,  7.7670e+01,  2.0498e+00,  0.0000e+00],
        [ 3.3012e+00,  4.0340e+01,  1.4580e-01,  0.0000e+00],
        [ 2.9526e+01,  6.5011e+01,  5.7588e-01,  0.0000e+00],
        [ 3.7855e+01,  7.0953e+01,  6.9073e-01,  0.0000e+00],
        [-2.0532e+00,  3.8026e+01,  2.7026e-01,  0.0000e+00],
        [ 3.7712e+01,  5.9681e+01,  2.0375e+01,  1.0000e+00],
        [ 8.5972e+00, -2.8851e+01,  1.7925e+01,  1.0000e+00],
        [ 6.2841e+00,  3.1319e+01,  2.0017e+01,  1.0000e+00],
        [-7.8481e+00,  7.3213e+01,  2.2176e+01,  1.0000e+00],
        [ 3.8192e+00,  4.1440e+01,  2.0297e+01,  1.0000e+00],
        [ 2.9870e+01,  

In [20]:
#Output
from groundtruth_generation.nuscenes_create_gt import edge_label_classes
print("edge_label_classes.different_instance:",edge_label_classes.different_instance)
print("edge_label_classes.same_instance:",edge_label_classes.same_instance)
print("edge_label_classes.new_instance:",edge_label_classes.new_instance)
print("Edge labels: one hot encoding",MotGraphList[0].graph_obj["edge_labels"])

edge_label_classes.different_instance: edge_label_classes.different_instance
edge_label_classes.same_instance: edge_label_classes.same_instance
edge_label_classes.new_instance: edge_label_classes.new_instance
Edge labels: one hot encoding tensor([[1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
 

In [12]:
# print("----------------------------------------\n")
# print(len(MotGraphList))
# print("----------------------------------------\n")
# print(MotGraphList[0])
# print("----------------------------------------\n")
# print(MotGraphList[0].graph_obj)
# print("----------------------------------------\n")
# print(len(MotGraphList[0].graph_obj))
# print("----------------------------------------\n")
# print(MotGraphList[0].graph_obj.keys)
# print("----------------------------------------\n")
# print("is_undirected() : ",MotGraphList[0].graph_obj.is_undirected())
# print("----------------------------------------\n")
# print(MotGraphList[0].graph_obj["x"][0:5])

# print("----------------------------------------\n")
# print('graph_dataframe["boxes_dict"][1][0] \n',MotGraphList[0].graph_dataframe["boxes_dict"][1][0])
# print("----------------------------------------\n")
# box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
# print("box_list: \n",box_list[0], 'center_list: \n', center_list[0])
# print("----------------------------------------\n")
# box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
# print("is same memory ?: \n", MotGraphList[0].graph_dataframe["boxes_dict"][1] is box_list )
# print("----------------------------------------\n")
# for i in range(3):
#     print(MotGraphList[0].graph_dataframe["boxes_list_all"][i] )
#     print(MotGraphList[0].graph_dataframe["centers_list_all"][i])
#     print("----------------------------------------\n")
# print("----------------------------------------\n")
# for i in range(3):
#     print(MotGraphList[0].graph_dataframe["boxes_list_all"][-i] )
#     print(MotGraphList[0].graph_dataframe["centers_list_all"][-i])
#     print("----------------------------------------\n")
# print("----------------------------------------\n")
# print("Edge Labels: ",MotGraphList[0].graph_obj["edge_labels"])
# print("----------------------------------------\n")
# print("Edge Labels: ",MotGraphList[-1].graph_obj["edge_labels"])
# print("----------------------------------------\n")

# print("Nodes shape: ",MotGraphList[0].graph_obj.x.shape)
# print("----------------------------------------\n")
# print("Edge_index : ",MotGraphList[0].graph_obj["edge_index"])
# print("----------------------------------------\n")

In [13]:
# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv

# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = GCNConv(3, 16)
#         self.conv2 = GCNConv(16, 2)
#         self.mlp = torch.nn.Linear(
#                     in_features=3, out_features=1, bias=True)

#     def forward(self, data):
#         x = data.x 
#         edge_index = data.edge_index
#         edge_feature =data.edge_attr

#         # x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         # x = self.conv2(x, edge_index)
#         x = x.float()
#         x = self.mlp(x)

#         return F.sigmoid(x)

In [14]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

graph_list = []
for graph in MotGraphList:
    graph_list.append(graph.graph_obj)
# print(len(graph_list))
# print(graph_list[:5])
train_loader = DataLoader(graph_list, batch_size=3,shuffle =False)

In [15]:
a = MotGraphList[0].graph_obj.edge_labels
print(a[:5])
print(a.shape)
b =a.argmax(dim=1)
print(b[:5])
print(b.shape)

c = MotGraphList[0].graph_obj.edge_attr.to("cuda:1")
print(c[:5])
print(c.shape)
c_2 = torch.zeros(c.shape[0],1).to("cuda:1")
d = torch.cat([c, c_2 ], dim= 1).to("cuda:1")
print(d[:5])
print(d.shape)

tensor([[1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0]], device='cuda:1', dtype=torch.uint8)
torch.Size([330, 3])
tensor([0, 1, 0, 1, 0], device='cuda:1')
torch.Size([330])
tensor([[0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]], device='cuda:1', dtype=torch.uint8)
torch.Size([330, 2])
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='cuda:1')
torch.Size([330, 3])


In [16]:
# from model.message_passing_module import Message_Passing_spatio_temporal

# message_passing_param_dict = {'node_feature_input_size': 4, 
#                                     "edge_feature_inpute_size": 2,
#                                     "node_feature_output_size":10,
#                                     "edge_feature_output_size":10,
#                                     }

# model = Message_Passing_spatio_temporal(
#                     in_channels = message_passing_param_dict["node_feature_input_size"],
#                     out_channels = "edge_feature_inpute_size"
#                     )
# model.to("cuda:1")

# print(next(model.parameters()).is_cuda)

# # datapoint = MotGraphList[0].graph_obj
# # output = model(datapoint)

# for data in train_loader:
#     temporal_edge_embeddings = model(data)

In [18]:
from model.mpn import MOTMPNet, EdgeModel, TimeAwareNodeModel, MetaLayer
from model.mlp import MLP
from torch_scatter import scatter_mean, scatter_max, scatter_add
import tqdm

node_feature_size = 4
edge_feature_size = 2
middle_feature_size = 10

edge_mlp = MLP(input_dim=2 * node_feature_size +edge_feature_size ,
                fc_dims=[middle_feature_size,edge_feature_size],dropout_p=None)
flow_in_mlp = MLP(input_dim=node_feature_size +edge_feature_size,
                fc_dims=[middle_feature_size],dropout_p=None)
flow_out_mlp = MLP(input_dim=node_feature_size +edge_feature_size, 
                fc_dims=[middle_feature_size],dropout_p=None)
node_mlp = MLP(input_dim=2* 10, 
                fc_dims=[node_feature_size],dropout_p=None)
node_agg_fn = lambda out, row, x_size: scatter_max(out, row, dim=0, dim_size=x_size)[0]

meta_model = MetaLayer(edge_model=EdgeModel(edge_mlp = edge_mlp),
            node_model=TimeAwareNodeModel(flow_in_mlp = flow_in_mlp,
                                        flow_out_mlp = flow_out_mlp,
                                        node_mlp = node_mlp,
                                        node_agg_fn = node_agg_fn))
str_device = "cuda:1"
meta_model.to(str_device)

optimizer = torch.optim.Adam(meta_model.parameters(), lr=0.0005)
crit = torch.nn.CrossEntropyLoss()

common_dtype = torch.float

loss_general = []
# for epoch in tqdm(range(10),desc="epoch"):
for epoch in range(10):
    loss_all = 0
    for graph_object_mini_batch in train_loader:
        x = graph_object_mini_batch.x.type(common_dtype)
        edge_index = graph_object_mini_batch.edge_index
        edge_feature = graph_object_mini_batch.edge_attr.type(common_dtype)
        temporal_edges_mask = graph_object_mini_batch.temporal_edges_mask

        x, edge_attr = meta_model(x,edge_index,edge_feature)
        #Modify Labels a bit 
        edge_attr = torch.cat([edge_attr, torch.zeros(edge_attr.shape[0],1).to("cuda:1")], dim= 1)
        edge_labels_one_hot = graph_object_mini_batch.edge_labels
        edge_labels_class_numbers = edge_labels_one_hot.argmax(dim=1)


        loss = crit(edge_attr, edge_labels_class_numbers)
        loss.backward()
        loss_all += graph_object_mini_batch.num_graphs * loss.item()
        optimizer.step()
        # print('loss',loss)
        # print("loss_all",loss_all)
    loss_normalized = loss_all / len(graph_list)
    loss_general.append(loss_normalized)
    print('epoch',epoch,"loss_normalized",loss_normalized)

epoch 0 loss_normalized 2.852040464813645
epoch 1 loss_normalized 2.112103082038261
epoch 2 loss_normalized 1.4118491991146191
epoch 3 loss_normalized 1.012095346644118
epoch 4 loss_normalized 0.9522434231397268
epoch 5 loss_normalized 1.0069509599659894
epoch 6 loss_normalized 1.0714520248206887
epoch 7 loss_normalized 1.0932143652761304
epoch 8 loss_normalized 1.0359797171644263
epoch 9 loss_normalized 0.9177280761100151


In [ ]:

# def train():
#     model.train()

#     loss_all = 0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         label = data.y.to(device)
#         loss = crit(output, label)
#         loss.backward()
#         loss_all += data.num_graphs * loss.item()
#         optimizer.step()
#     return loss_all / len(graph_list)

# from tqdm import tqdm

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

# model = Net().to(device)
# print(next(model.parameters()).is_cuda)
# # print(model.parameters())

# optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# crit = torch.nn.BCELoss()

# print(train_loader)

# loss_general = []
# for epoch in tqdm(range(4),desc="epoch"):
#     model.train()
#     loss_all = 0
#     for data in train_loader:
#         optimizer.zero_grad()
#         output = model(data)
#         #Generate random labels for testing pipeline
#         # generate binary labels 
#         probabilities_labels = torch.empty(output.shape).uniform_(0, 1).to(device)
#         node_labels_binary = torch.bernoulli(probabilities_labels)
#         loss = crit(output, node_labels_binary)
#         loss.backward()
#         loss_all += data.num_graphs * loss.item()
#         optimizer.step()
#     loss_normalized =loss_all / len(graph_list)
#     loss_general.append(loss_normalized)
#     print('epoch',epoch,'loss_general:',loss_normalized)

TypeError: edge_updater() takes 2 positional arguments but 4 were given